# MAF User manual

## 1. DataSet

### 1.1. Read from file (csv, tsv, npy)

#### 1.1.1. Read npy

In [1]:
# Make a sample npy file
import numpy as np

feature = np.random.random((500, 2))
target = np.random.randint(0, 2, size=500)
bias = np.random.randint(0, 2, size=500)

whole = np.column_stack((feature, bias, target))
np.save('sample.npy', whole)

In [2]:
# Use RawDataSet
from DataSet import RawDataSet

## Required parameters
#    - target column index (int) : target_col_idx
#    - bias column index (int) : bias_col_idx
## Optional parameters
#    - prediction column index (int) : pred_col_idx

dataset = RawDataSet('sample.npy', target_col_idx=3, bias_col_idx=2)

2022-11-28 17:57:38.188374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/e/MAF_API/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-28 17:57:38.188410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
print("Shape of feature: ", dataset.feature.shape)
print('Shape of feature_only: ', dataset.feature_only.shape)
print('Shape of target: ', dataset.target.shape)
print('Shape of bias: ', dataset.bias.shape)

Shape of feature:  (500, 3)
Shape of feature_only:  (500, 2)
Shape of target:  (500,)
Shape of bias:  (500,)


#### 1.1.2. Read table

In [4]:
# Check the sample data
import pandas as pd

df = pd.read_table('./CelebA_samples.csv', sep=',')
df.head()

,Heavy_Makeup,Male,0,1,2,3,4,5,6,7,...,12278,12279,12280,12281,12282,12283,12284,12285,12286,12287
0,-1,-1,255,255,253,254,254,251,253,254,...,0,0,0,0,0,0,0,0,0,0
1,-1,-1,32,61,70,14,32,43,16,22,...,4,2,2,2,1,1,1,2,2,2
2,1,-1,27,23,25,13,11,12,4,3,...,16,56,24,15,46,24,16,38,21,10
3,1,-1,30,31,25,38,39,33,52,53,...,13,49,30,17,67,45,31,85,58,40
4,-1,-1,54,39,10,54,39,10,54,39,...,21,12,17,18,10,16,16,14,16,18


In [5]:
# Read the sample data using RawDataSet
dataset = RawDataSet('./CelebA_samples.csv', header=0, seperator=',', 
                     target_col_name='Male', bias_col_name='Heavy_Makeup', cate_cols=['Male', 'Heavy_Makeup'])

In [6]:
print("Shape of feature: ", dataset.feature.shape)
print('Shape of feature_only: ', dataset.feature_only.shape)
print('Shape of target: ', dataset.target.shape)
print('Shape of bias: ', dataset.bias.shape)

Shape of feature:  (500, 12289)
Shape of feature_only:  (500, 12288)
Shape of target:  (500,)
Shape of bias:  (500,)


### 1.2. aifData

In [7]:
from DataSet import aifData

df = pd.read_table('./CelebA_samples.csv', sep=',')

aif_data = aifData(df=df, label_name='Male', favorable_classes=[1],
                  protected_attribute_names=['Heavy_Makeup'], privileged_classes=[[1]])

## 2. Metrics

### 2.1. Metrics for dataset

In [8]:
from Metric import DataMetric

metric = DataMetric(aif_data, privilege=[{'Heavy_Makeup': 1}], unprivilege=[{'Heavy_Makeup': -1}])

In [9]:
print('Negatives N', metric.num_negative())
print('Positives N', metric.num_positive())
print('Base rate', metric.base_rate())
print('Statistical parity difference', metric.statistical_parity_difference())
print('Consistency', metric.consistency())

Negatives N 304
Positives N 196
Base rate 0.392
Statistical parity difference 0.6468646864686468
Consistency 0.3735999999999997


### 2.2. Metrics for prediction

In [10]:
# Split the data
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, train_size=0.7, shuffle=True, random_state=777)

In [11]:
train_aif = aifData(df=train_df, label_name='Male', favorable_classes=[1],
                  protected_attribute_names=['Heavy_Makeup'], privileged_classes=[[1]])
test_aif = aifData(df=test_df, label_name='Male', favorable_classes=[1],
                  protected_attribute_names=['Heavy_Makeup'], privileged_classes=[[1]])

In [12]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
model.fit(train_aif.features, train_aif.labels.ravel())

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.


SGDClassifier(max_iter=5)

In [13]:
prediction = model.predict(test_aif.features)

In [14]:
from Metric import ClassificationMetric

clsMetric = ClassificationMetric(test_aif, privilege=[{'Heavy_Makeup': 1}], unprivilege=[{'Heavy_Makeup': -1}],
                                prediction_vector=prediction, target_label_name='Male')

In [15]:
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.9672131147540983, 'TNR': 0.39325842696629215, 'FPR': 0.6067415730337079, 'FNR': 0.03278688524590164, 'PPV': 0.5221238938053098, 'NPV': 0.9459459459459459, 'FDR': 0.4778761061946903, 'FOR': 0.05405405405405406, 'ACC': 0.6266666666666667}
Error rate:  0.3733333333333333
Average odds difference:  0.5706755228942906
Average absolute odds difference:  0.5706755228942906
Selection rate:  0.7533333333333333
Disparate impact:  1.616161616161616
Statistical parity difference:  0.3388888888888888
Generalized entropy index:  0.0697970787177727
Theil index:  0.07296113877660586
Equal opportunity difference:  0.9672131147540983


## 3. Algorithms

### 3.1. AIF360 algorithms

In [16]:
from Algorithms.Preprocessing import RW

rw = RW(unprivileged_groups=[{'Heavy_Makeup': -1}], privileged_groups = [{'Heavy_Makeup': 1}])
tfm_train_aif = rw.fit_transform(train_aif)

divide by zero encountered in double_scalars


In [17]:
# Check the bias
model.fit(tfm_train_aif.features, tfm_train_aif.labels.ravel())

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.


SGDClassifier(max_iter=5)

In [18]:
tfm_predict = model.predict(test_aif.features)
clsMetric = ClassificationMetric(test_aif, privilege=[{'Heavy_Makeup': 1}], unprivilege=[{'Heavy_Makeup': -1}],
                                prediction_vector=tfm_predict, target_label_name='Male')
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.819672131147541, 'TNR': 0.651685393258427, 'FPR': 0.34831460674157305, 'FNR': 0.18032786885245902, 'PPV': 0.6172839506172839, 'NPV': 0.8405797101449275, 'FDR': 0.38271604938271603, 'FOR': 0.15942028985507245, 'ACC': 0.72}
Error rate:  0.28
Average odds difference:  0.48397399660825324
Average absolute odds difference:  0.48397399660825324
Selection rate:  0.54
Disparate impact:  2.3333333333333335
Statistical parity difference:  0.39999999999999997
Generalized entropy index:  0.10207612456747402
Theil index:  0.12763171113256225
Equal opportunity difference:  0.819672131147541


### 3.2. S.O.T.A. Algorithms

#### 3.2.1. FairBatch

In [19]:
# Prepare raw dataset
#dataset = RawDataSet('CelebA_samples.csv', target_col_name='Male', bias_col_name='Heavy_Makeup')
dataset = RawDataSet('sample.npy', target_col_idx=3, bias_col_idx=2)

In [20]:
from Algorithms.sota import FairBatch
# Get prediction
fb_pred = FairBatch.train(kaif_raw_dataset=dataset, batch_size=4, alpha=2, target_fairness="eqopp")

Trained on [[[  cpu  ]]] device.
Epoch [0] || Average loss : 0.5789919444918632
Epoch [10] || Average loss : 0.47548798948526383
Epoch [20] || Average loss : 0.4754349333047867
Epoch [30] || Average loss : 0.4737826558947563
Epoch [40] || Average loss : 0.4731024914979935
Epoch [50] || Average loss : 0.47203448742628096
Epoch [60] || Average loss : 0.4712680643796921
Epoch [70] || Average loss : 0.47039511740207673
Epoch [80] || Average loss : 0.46937511473894117
Epoch [90] || Average loss : 0.4684423300623894
Epoch [100] || Average loss : 0.4675027135014534
Epoch [110] || Average loss : 0.4665996280312538
Epoch [120] || Average loss : 0.4654026520252228
Epoch [130] || Average loss : 0.4658711650967598
Epoch [140] || Average loss : 0.4643395459651947
Epoch [150] || Average loss : 0.46237372130155563
Epoch [160] || Average loss : 0.46037332236766815
Epoch [170] || Average loss : 0.4605739066004753
Epoch [180] || Average loss : 0.4542435497045517
Epoch [190] || Average loss : 0.450987484

In [21]:
arr = np.load('sample.npy')
df = pd.DataFrame(arr, columns=['Attr1', 'Attr2', 'Bias', 'Target'])
aif_data = aifData(df=df, label_name='Target', favorable_classes=[1],
                  protected_attribute_names=['Bias'], privileged_classes=[[1]])
clsMetric = ClassificationMetric(aif_data, privilege=[{'Bias': 1}], unprivilege=[{'Bias': 0}],
                                prediction_vector=fb_pred, target_label_name='Target')

In [22]:
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.43824701195219123, 'TNR': 0.6987951807228916, 'FPR': 0.30120481927710846, 'FNR': 0.5617529880478087, 'PPV': 0.5945945945945946, 'NPV': 0.5523809523809524, 'FDR': 0.40540540540540543, 'FOR': 0.44761904761904764, 'ACC': 0.568}
Error rate:  0.43200000000000005
Average odds difference:  -0.7039382373351839
Average absolute odds difference:  0.7039382373351839
Selection rate:  0.37
Disparate impact:  0.0
Statistical parity difference:  -0.7034220532319392
Generalized entropy index:  0.27512794920257405
Theil index:  0.381130562211169
Equal opportunity difference:  -0.8396946564885496


#### 3.2.2. Fair Feature Distillation

In [23]:
rds = RawDataSet('./CelebA_samples.csv', header=0, seperator=',', 
                 target_col_name='Male', bias_col_name='Heavy_Makeup', cate_cols=['Male', 'Heavy_Makeup'])

In [24]:
from Algorithms.sota.FairFeatureDistillation import FFD

ffd = FFD(rds, n_epoch=20, batch_size=20, learning_rate=0.01, image_shape=[3, 64, 64])

Trained on [[[  cpu  ]]] device.


In [25]:
ffd.train_teacher()

Train teacher start
Epoch [1/20], Batch [0/20], Loss 0.0
Epoch [1/20], Batch [10/20], Loss 0.0
Epoch [2/20], Batch [0/20], Loss 0.0
Epoch [2/20], Batch [10/20], Loss 0.0
Epoch [3/20], Batch [0/20], Loss 0.0
Epoch [3/20], Batch [10/20], Loss 0.0
Epoch [4/20], Batch [0/20], Loss 0.0
Epoch [4/20], Batch [10/20], Loss 0.0
Epoch [5/20], Batch [0/20], Loss 0.0
Epoch [5/20], Batch [10/20], Loss 0.0
Epoch [6/20], Batch [0/20], Loss 0.0
Epoch [6/20], Batch [10/20], Loss 0.0
Epoch [7/20], Batch [0/20], Loss 0.0
Epoch [7/20], Batch [10/20], Loss 0.0
Epoch [8/20], Batch [0/20], Loss 0.0
Epoch [8/20], Batch [10/20], Loss 0.0
Epoch [9/20], Batch [0/20], Loss 0.0
Epoch [9/20], Batch [10/20], Loss 0.0
Epoch [10/20], Batch [0/20], Loss 0.0
Epoch [10/20], Batch [10/20], Loss 0.0
Epoch [11/20], Batch [0/20], Loss 0.0
Epoch [11/20], Batch [10/20], Loss 0.0
Epoch [12/20], Batch [0/20], Loss 0.0
Epoch [12/20], Batch [10/20], Loss 0.0
Epoch [13/20], Batch [0/20], Loss 0.0
Epoch [13/20], Batch [10/20], Loss 0

In [26]:
ffd.train_student()

Train student start
Epoch [1/20], Batch [0/20], Loss 0.0431215763092041
Epoch [1/20], Batch [10/20], Loss 0.03219097852706909
Epoch [2/20], Batch [0/20], Loss 0.03266716003417969
Epoch [2/20], Batch [10/20], Loss 0.022750377655029297
Epoch [3/20], Batch [0/20], Loss 0.02060335874557495
Epoch [3/20], Batch [10/20], Loss 0.02368074655532837
Epoch [4/20], Batch [0/20], Loss 0.021546363830566406
Epoch [4/20], Batch [10/20], Loss 0.02171158790588379
Epoch [5/20], Batch [0/20], Loss 0.019782721996307373
Epoch [5/20], Batch [10/20], Loss 0.019554615020751953
Epoch [6/20], Batch [0/20], Loss 0.01796180009841919
Epoch [6/20], Batch [10/20], Loss 0.01750248670578003
Epoch [7/20], Batch [0/20], Loss 0.017918944358825684
Epoch [7/20], Batch [10/20], Loss 0.018960893154144287
Epoch [8/20], Batch [0/20], Loss 0.01612168550491333
Epoch [8/20], Batch [10/20], Loss 0.015807807445526123
Epoch [9/20], Batch [0/20], Loss 0.016983866691589355
Epoch [9/20], Batch [10/20], Loss 0.016386210918426514
Epoch [10

In [27]:
pred = ffd.evaluation()

Evaluation finished.


In [28]:
test_X = ffd.test_dataset.X.reshape(100, -1).cpu().detach().numpy()
test_y = ffd.test_dataset.y.cpu().detach().numpy()
test_z = ffd.test_dataset.z.cpu().detach().numpy()

In [29]:
# Measuring bias
df = pd.DataFrame(test_X)
df['Male'] = test_y
df['Heavy_Makeup'] = test_z
df.head()

,0,1,2,3,4,5,6,7,8,9,...,12280,12281,12282,12283,12284,12285,12286,12287,Male,Heavy_Makeup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,3.0,27.0,2.0,1.0,5.0,0.0,0.0,0,0
1,97.0,63.0,35.0,98.0,64.0,36.0,101.0,66.0,37.0,106.0,...,206.0,213.0,221.0,207.0,213.0,221.0,208.0,212.0,0,0
2,32.0,24.0,13.0,31.0,23.0,12.0,31.0,23.0,12.0,30.0,...,11.0,24.0,86.0,100.0,119.0,124.0,144.0,165.0,0,0
3,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,64.0,44.0,65.0,64.0,44.0,65.0,64.0,43.0,0,0
4,77.0,51.0,60.0,72.0,51.0,56.0,65.0,52.0,52.0,62.0,...,144.0,103.0,213.0,179.0,136.0,204.0,177.0,132.0,0,0


In [30]:
aif_data = aifData(df=df, label_name='Male', favorable_classes=[1],
                  protected_attribute_names=['Heavy_Makeup'], privileged_classes=[[1]])

In [31]:
clsMetric = ClassificationMetric(aif_data, privilege=[{'Heavy_Makeup': 1}], unprivilege=[{'Heavy_Makeup': 0}],
                                prediction_vector=pred, target_label_name='Male')
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.0, 'TNR': 1.0, 'FPR': 0.0, 'FNR': 0.0, 'PPV': 0.0, 'NPV': 1.0, 'FDR': 0.0, 'FOR': 0.0, 'ACC': 1.0}
Error rate:  0.0
Average odds difference:  0.0
Average absolute odds difference:  0.0
Selection rate:  0.0
Disparate impact:  0
Statistical parity difference:  0.0
Generalized entropy index:  0.0
Theil index:  0.0
Equal opportunity difference:  0.0


#### 3.2.3. Fairness VAE

In [32]:
from Algorithms.sota.FairnessVAE import fVAEDataset, FairnessVAE

Train on [[[  cpu  ]]] device.


In [33]:
rds = RawDataSet('./CelebA_samples.csv', header=0, seperator=',', 
                 target_col_name='Male', bias_col_name='Heavy_Makeup', cate_cols=['Male', 'Heavy_Makeup'])

In [34]:
fvae = FairnessVAE(dataset = rds,
                  z_dim = 10,
                  batch_size = 20,
                  num_epochs = 10,
                  image_shape = (3, 64, 64))

In [35]:
fvae.train_upstream()

### Training upstream start
Epoch [001]   VAE loss: 8216.741   Discriminator loss: 2.444
Epoch [002]   VAE loss: 8172.522   Discriminator loss: 79.178
Epoch [003]   VAE loss: 8106.138   Discriminator loss: 1383.587
Epoch [004]   VAE loss: 7858.494   Discriminator loss: 12996.923
Epoch [005]   VAE loss: 5632.970   Discriminator loss: 175547.078
Epoch [006]   VAE loss: 1311.787   Discriminator loss: 1959365.375
Epoch [007]   VAE loss: 344.635   Discriminator loss: 4482038.500
Epoch [008]   VAE loss: -287.111   Discriminator loss: 15891211.000
Epoch [009]   VAE loss: -2691.147   Discriminator loss: 89651440.000
Epoch [010]   VAE loss: -12560.896   Discriminator loss: 407197440.000
### Upstream training done.


In [36]:
fvae.train_downstream()

### Downstream training start.
Epoch [001]   VAE loss: -8005843.000   Discriminator loss: 0.000
Epoch [002]   VAE loss: -7971678.500   Discriminator loss: 0.000
Epoch [003]   VAE loss: -7812424.000   Discriminator loss: 0.000
Epoch [004]   VAE loss: -8093974.500   Discriminator loss: 0.000
Epoch [005]   VAE loss: -8115528.000   Discriminator loss: 0.000
Epoch [006]   VAE loss: -7988520.000   Discriminator loss: 0.000
Epoch [007]   VAE loss: -8178860.000   Discriminator loss: 0.000
Epoch [008]   VAE loss: -8053573.000   Discriminator loss: 0.000
Epoch [009]   VAE loss: -8034507.000   Discriminator loss: 0.000
Epoch [010]   VAE loss: -7963781.500   Discriminator loss: 0.000
### Downstream training done.


In [37]:
pred = fvae.evaluation()

### Evaluation start.
### Evaluation done.


In [38]:
feature_set = fvae.test_dataset.feature.cpu().detach().numpy()
target_set = fvae.test_dataset.target.cpu().detach().numpy()
bias_set = fvae.test_dataset.bias.cpu().detach().numpy()

df = pd.DataFrame(feature_set)
df['Male'] = target_set
df['Heavy_Makeup'] = bias_set

aif_data = aifData(df=df, label_name='Male', favorable_classes=[1],
                  protected_attribute_names=['Heavy_Makeup'], privileged_classes=[[1]])

clsMetric = ClassificationMetric(aif_data, privilege=[{'Heavy_Makeup': 1}], unprivilege=[{'Heavy_Makeup': 0}],
                                prediction_vector=pred, target_label_name='Male')
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.0, 'TNR': 1.0, 'FPR': 0.0, 'FNR': 0.0, 'PPV': 0.0, 'NPV': 1.0, 'FDR': 0.0, 'FOR': 0.0, 'ACC': 1.0}
Error rate:  0.0
Average odds difference:  0.0
Average absolute odds difference:  0.0
Selection rate:  0.0
Disparate impact:  0
Statistical parity difference:  0.0
Generalized entropy index:  0.0
Theil index:  0.0
Equal opportunity difference:  0.0


#### 3.2.4. Kernel Density Estimation

In [39]:
# Data
rds = RawDataSet('sample.npy', target_col_idx=3, bias_col_idx=2)

In [40]:
from Algorithms.sota.KernelDensityEstimator import KDEmodel

kde = KDEmodel(rawdata=rds, fairness_notion='DP', batch_size=32, n_epoch=20, learning_rate=0.01)

Train on [[[  cpu  ]]] device.


In [41]:
kde.train()

Train model start.
Train model done.tch [12/12], Cost: 0.5919


In [42]:
pred = kde.evaluation()

Evaluation start.
Evaluation done.


In [43]:
feature = kde.test_data.X.cpu().detach().numpy()
target = kde.test_data.y.cpu().detach().numpy()
bias = kde.test_data.z.cpu().detach().numpy()

In [44]:
df = pd.DataFrame(feature)
df['Target'] = target
df['Bias'] = bias
df.head()

,0,1,2,Target,Bias
0,0.758807,0.600065,1.0,1,1
1,0.554044,0.843816,1.0,0,1
2,0.350958,0.762025,0.0,1,0
3,0.503366,0.612485,0.0,0,0
4,0.461010,0.550399,1.0,1,1


In [45]:
aif_data = aifData(df=df, label_name='Target', favorable_classes=[1],
                  protected_attribute_names=['Bias'], privileged_classes=[[1]])
clsMetric = ClassificationMetric(aif_data, privilege=[{'Bias': 1}], unprivilege=[{'Bias': 0}],
                                prediction_vector=pred, target_label_name='Target')
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.5714285714285714, 'TNR': 0.43137254901960786, 'FPR': 0.5686274509803921, 'FNR': 0.42857142857142855, 'PPV': 0.49122807017543857, 'NPV': 0.5116279069767442, 'FDR': 0.5087719298245614, 'FOR': 0.4883720930232558, 'ACC': 0.5}
Error rate:  0.5
Average odds difference:  0.2914910600255427
Average absolute odds difference:  0.2914910600255427
Selection rate:  0.57
Disparate impact:  1.6669750231267346
Statistical parity difference:  0.28944199116820557
Generalized entropy index:  0.21159122085047996
Theil index:  0.2952846669423606
Equal opportunity difference:  0.21724137931034476


#### 3.2.5. Learning from Fairness

In [46]:
from DataSet import RawDataSet

# Data
rds = RawDataSet('CelebA_samples.csv', target_col_name='Male', bias_col_name='Heavy_Makeup')
rds.target = (rds.target > 0)*1
rds.bias = (rds.bias > 0)*1

In [47]:
from Algorithms.sota.LearningFromFairness import LfFmodel

lfm = LfFmodel(rawdata=rds, n_epoch=20, batch_size=16, learning_rate=0.01, image_shape=(3, 64, 64))

Train on [[[  cpu  ]]] device.


In [48]:
lfm.train()

Training start
Epoch 0/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 1/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 2/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 3/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 4/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 5/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 6/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 7/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 8/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 9/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 10/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 11/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 12/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 13/20 :  Loss nan | valid_accuracy_b nan | valid_accuracy_d nan
Epoch 14/20 :  

In [49]:
accs, bias_list, label_list, predict_list = lfm.evaluate(lfm.model_d)

In [50]:
feature = lfm.test_dataset.X.cpu().detach().numpy()
bias = lfm.test_dataset.z.cpu().detach().numpy()
target = lfm.test_dataset.y.cpu().detach().numpy()

In [51]:
df = pd.DataFrame(feature)
df['Target'] = target
df['Bias'] = bias
df.head()

,0,1,2,3,4,5,6,7,8,9,...,12280,12281,12282,12283,12284,12285,12286,12287,Target,Bias
0,1.0,2.0,4.0,1.0,2.0,4.0,1.0,2.0,4.0,1.0,...,23.0,18.0,30.0,31.0,26.0,26.0,31.0,26.0,0,1
1,2.0,2.0,4.0,2.0,2.0,4.0,2.0,2.0,4.0,2.0,...,101.0,73.0,125.0,101.0,76.0,152.0,130.0,106.0,0,0
2,117.0,117.0,119.0,120.0,120.0,122.0,124.0,124.0,126.0,128.0,...,123.0,89.0,180.0,120.0,86.0,174.0,114.0,79.0,0,1
3,192.0,195.0,214.0,192.0,195.0,214.0,192.0,195.0,214.0,192.0,...,133.0,160.0,138.0,136.0,146.0,149.0,138.0,142.0,0,1
4,254.0,253.0,190.0,255.0,253.0,193.0,255.0,254.0,196.0,254.0,...,160.0,239.0,125.0,173.0,193.0,169.0,182.0,167.0,1,0


In [52]:
aif_data = aifData(df=df, label_name='Target', favorable_classes=[1],
                  protected_attribute_names=['Bias'], privileged_classes=[[1]])
clsMetric = ClassificationMetric(aif_data, privilege=[{'Bias': 1}], unprivilege=[{'Bias': 0}],
                                prediction_vector=pred, target_label_name='Target')
print('Performance measure: ', clsMetric.performance)
print('Error rate: ', clsMetric.error_rate())
print('Average odds difference: ', clsMetric.average_odds_difference())
print('Average absolute odds difference: ', clsMetric.average_abs_odds_difference())
print('Selection rate: ', clsMetric.selection_rate())
print('Disparate impact: ', clsMetric.disparate_impact())
print('Statistical parity difference: ', clsMetric.statistical_parity_difference())
print('Generalized entropy index: ', clsMetric.generalized_entropy_index())
print('Theil index: ', clsMetric.theil_index())
print('Equal opportunity difference: ', clsMetric.equal_opportunity_difference())

Performance measure:  {'TPR': 0.6136363636363636, 'TNR': 0.4642857142857143, 'FPR': 0.5357142857142857, 'FNR': 0.38636363636363635, 'PPV': 0.47368421052631576, 'NPV': 0.6046511627906976, 'FDR': 0.5263157894736842, 'FOR': 0.3953488372093023, 'ACC': 0.53}
Error rate:  0.47
Average odds difference:  0.2805023923444976
Average absolute odds difference:  0.33313397129186606
Selection rate:  0.57
Disparate impact:  1.0506912442396312
Statistical parity difference:  0.028013582342954146
Generalized entropy index:  0.17742188111833354
Theil index:  0.24582511801554485
Equal opportunity difference:  0.6136363636363636
